<img src="EEGRIDAnalytics.png">

In [ ]:
import pandas as pd
import numpy as np
import pyodbc


import ipywidgets as widgets
import json
#from ipyleaflet import Map, GeoJSON

import datetime as dt
import tkinter.messagebox as messagebox
import numpy as np
from ipyleaflet import *
from ipywidgets import *


In [16]:
ee_basemaps={}
ee_basemaps['ARCGIS_Satelite'] = TileLayer(url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}')

# Loops through all ipyleaflet basemaps
for item in basemaps.values():
    try:
        name = item['name']
        basemap = 'basemaps.{}'.format(name)
        ee_basemaps[name] = basemap_to_tiles(eval(basemap))
    except:
        for sub_item in item:
            name = item[sub_item]['name']
            basemap = 'basemaps.{}'.format(name)
            basemap = basemap.replace('Mids', 'Modis')
            ee_basemaps[name] = basemap_to_tiles(eval(basemap))
list_maps = list(ee_basemaps.keys())
list_maps_2 = list_maps[0:2]

In [18]:
def change_map_layer(change):
    print('layer change start')
    basemap_name = change['new']
    list_classes1 = [m.layers[jj].__class__.__name__ for jj in range(0,len(m.layers))]
    if len(list_classes1)==1:
        m.add_layer(ee_basemaps[basemap_name])
    else:
        list_classes2 = [m.layers[jj].__class__.__name__ for jj in range(1,len(m.layers))]
        if 'TileLayer' in list_classes2:
            old_basemap1 = m.layers[list_classes2.index('TileLayer')+1]
            m.substitute_layer(old_basemap1, ee_basemaps[basemap_name])
        else:
            m.add_layer(ee_basemaps[basemap_name])
            

In [48]:
df = pd.read_csv('school_data.csv')

In [61]:
zipcode_list = df['properties.ZIP'].unique()

In [69]:
def zipcode_control(change):
    print(change.new)
    if change.new != None:
        list_classes_BC = [m.layers[kk].__class__.__name__ for kk in range(0,len(m.layers))]
        for items in list_classes_BC:
            if items =='Marker':
                layer_BN = m.layers[list_classes_BC.index(items)]
                m.remove_layer(layer_BN)
                
        flag = 1
        df88_1=df[df['properties.ZIP'] == change.new]
#         df88_1 = df[df['properties.ZIP'] == 75253]
        for kk in range(len(df88_1)):
            x = df88_1['Latitude'].iloc[kk]
            y = df88_1['Longtitude'].iloc[kk]
        #     bus_fp = df88_1['PSSE New BUS'].iloc[0] 

            Name = df88_1['properties.Name'].iloc[kk]
        #     print(PSSE_New_BUS)
        #     PSSE_New_BUS = str(PSSE_New_BUS)
            CITY = df88_1['properties.CITY'].iloc[kk]

            SCHOOLNAME = df88_1['properties.SCHOOLNAME'].iloc[kk]
            ADDRESS = df88_1['properties.ADDRESS'].iloc[kk]
        #     County = str(County)


            marker = Marker(location = (x,y),draggable=False)

            message2 = HTML()
            message2.value ="Name: "+str(Name)+'<br>'+"CITY "+str(CITY)\
                            +'<br>'+'SCHOOLNAME: '+str(SCHOOLNAME)+'<br>'+'ADDRESS: '+str(ADDRESS)

            marker.popup = message2



            m.add_layer(marker)
        #     m.center = (x,y)
#             m.zoom = 18


In [70]:
center=(31.432,-102.4377)
#print(center)
m = Map(center=center, zoom_control=False, zoom=5,inertia = True, max_zoom = 30,scroll_wheel_zoom = True, inertia_max_speed = 1500, preferCanvas = True)
m.layout.height='500px'

full_screen = FullScreenControl(position='bottomleft')
m.add_control(full_screen)
m.add_control(ZoomControl(position='bottomleft'))

dd1 = widgets.Dropdown(options=zipcode_list,value=None,description='Zipcode:',layout=Layout(width='250px'))
dd1.observe(zipcode_control, names="value")
widget_control2 = WidgetControl(widget=dd1, position='topleft')
m.add_control(widget_control2)


Map_layer = widgets.Dropdown(options=list_maps_2,value=None,description='BaseMap:')
Map_layer.observe(change_map_layer, 'value')
base_layer = WidgetControl(widget=Map_layer, position='topright')
m.add_control(base_layer)

m

Map(center=[31.432, -102.4377], controls=(AttributionControl(options=['position', 'prefix'], position='bottomr…

In [67]:
with open('DISD_Campuses.geojson', 'r') as f:
    data = json.load(f)
features = data['features']
marker_c = []
for i in range(len(features)):
    location=(features[i]['geometry']['coordinates'][1],features[i]['geometry']['coordinates'][0])
    Name = features[i]['properties']['Name']
#     print(PSSE_New_BUS)
#     PSSE_New_BUS = str(PSSE_New_BUS)
    CITY = features[i]['properties']['CITY']

    SCHOOLNAME = features[i]['properties']['SCHOOLNAME']
    ADDRESS = features[i]['properties']['ADDRESS']
#     County = str(County)

    marker = Marker(location=location,draggable=False)
    message2 = HTML()
    message2.value ="Name: "+str(Name)+'<br>'+"CITY "+str(CITY)\
                    +'<br>'+'SCHOOLNAME: '+str(SCHOOLNAME)+'<br>'+'ADDRESS: '+str(ADDRESS)
    
    marker.icon = AwesomeIcon(name='battery',marker_color='red',icon_color='black', spin=False)

    marker.popup = message2
    marker_c.append(marker)
marker_cluster = MarkerCluster(markers =marker_c)   
m.add_layer(marker_cluster)